In [ ]:
#Keven Disen 111433335
#Final Project 2 Question 4

In [1]:
#pip install scikit-image

In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dropout
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np






Bad key "text.kerning_factor" on line 4 in
/Users/keven/opt/anaconda3/envs/tf/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
df=pd.read_csv('test.csv')
emotionsTest=df.emotion
images_flattened = df.pixels

# OK, now process all images to make all of them into numpy array to store 48x48 images
image=np.array(pd.Series(images_flattened[0].split()).map(lambda x: int(x))).reshape(1,48,48)

for i in range(1,len(images_flattened)):
    image_next = np.array(pd.Series(images_flattened[i].split()).map(lambda x: int(x))).reshape(1,48,48)
    image=np.vstack([image,image_next])
    
Xtest = image

In [4]:
Xtrain = np.load('np_training_data.npy')
df=pd.read_csv('facial_expression.csv') 
emotions=df.emotion
ytrain = np.array(emotions)
Xtrain.shape

(26859, 48, 48)

In [5]:
image_size = [48,48]

In [6]:
vgg = VGG16(input_shape= image_size+[3],weights='imagenet',include_top=False)

In [7]:
x = vgg.output
x = GlobalAveragePooling2D()(x) #Global average pooling operation for spatial data.
x = Dense(1024,activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(6,activation='softmax')(x) # now finally categorized to two groups


In [8]:
model = Model(inputs = vgg.input,outputs=preds)
vgg.input
vgg

In [ ]:
from keras.utils import to_categorical
from skimage.color import gray2rgb


X_train_=gray2rgb(Xtrain) 
X_test_ = gray2rgb(Xtest)

X_train_norm2 = X_train_ / 255
X_test_norm2 = X_test_ / 255


y_train_2 = ytrain.reshape((26859,1))

y_train_catg= to_categorical(y_train_2, num_classes=6)
y_test_catg = to_categorical(emotionsTest, num_classes=6)

predictions = model.predict(X_test_norm2)

# We use the same plotting method to see the confusion matrix
y_pred = [np.argmax(predictions[num]) for num in range(len(emotionsTest))]

from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(emotionsTest,y_pred)  # put true values on the left and predicted value on the right

import pandas as pd
import seaborn as sns

confusion_df = pd.DataFrame(conf_mat)
figure = plt.figure(figsize=(7,6))
axes = sns.heatmap(confusion_df, annot=True,annot_kws={"size": 8},
                  cmap=plt.cm.nipy_spectral_r)

axes.set_ylim(sorted(axes.get_xlim(), reverse=True))



In [ ]:
#It is already trained
for layer in vgg.layers: 
    layer.trainable = False
model.compile(optimizer='Adam', loss='binary_crossentropy',
              metrics=['accuracy'])




In [ ]:
model.fit(X_train_, y_train_catg, epochs=5, batch_size=32, validation_split=0.2)  # use 20% data for validation of the model

#Kernel kept dying and I tried using online notebooks, 
#but it said the files "facialexpression.csv" and "numpy_trainind_data" were too big

Train on 21487 samples, validate on 5372 samples
Epoch 1/5
